In [9]:
##### ALL IMPORTS ######

import os
import re
import requests
import pandas as pd
import json

## FUNCTION takes all files in the folder location
## Table returns: cik, filing_type,	filename, conformed_period_of_report

def parse_filing_folder(folder_path):
    rows = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") and ("10-Q" in filename or "10-K" in filename):
            # Extract CIK and filing type from the filename
            cik_match = re.search(r'edgar_data_(\d+)_', filename)
            type_match = re.search(r'10-[QK]', filename)

            cik = cik_match.group(1) if cik_match else None
            filing_type = type_match.group(0) if type_match else None

            # Full path to file
            file_path = os.path.join(folder_path, filename)

            # Read a portion of the file (not entire thing to save time)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read(5000)  # Read first 5000 characters
                    # Extract CONFORMED PERIOD OF REPORT
                    period_match = re.search(r'CONFORMED PERIOD OF REPORT:\s*(\d{8})', content)
                    period = period_match.group(1) if period_match else None
            except Exception as e:
                period = None

            rows.append({
                'cik': cik,
                'filing_type': filing_type,
                'filename': filename,
                'conformed_period_of_report': period
            })

    return pd.DataFrame(rows)


## call fucntion
folder_path = "Project" #path will need to be changed
df = parse_filing_folder(folder_path)



#ticker_path = "company_tickers.json"
def tickers(ticker_path):
    # Load JSON data
    with open(ticker_path, 'r') as f:
        ticker_data = json.load(f)
    
    # Convert JSON structure to DataFrame
    fields = ticker_data['fields']
    records = ticker_data['data']
    tickers_df = pd.DataFrame(records, columns=fields)
    
    # Normalize CIKs in both DataFrames to 10-digit strings
    tickers_df['cik'] = tickers_df['cik'].astype(str).str.zfill(10)
    df['cik'] = df['cik'].astype(str).str.zfill(10)
    
    # Rename for clarity
    tickers_df.rename(columns={'name': 'company_name'}, inplace=True)
    
    # Merge your df with the ticker info
    df2 = df.merge(tickers_df, on='cik', how='left')

    ### set to date time format
    df2['conformed_period_of_report'] = pd.to_datetime(df2['conformed_period_of_report'])
    ### remove the day to join on next table
    df2['date_clean'] = df2['conformed_period_of_report'].dt.to_period('M').astype(str)

    
    return df2

ticker_path = "company_tickers.json"  #path will need to be changed
df2 = tickers(ticker_path)


### finds all companys in the S&P by ticker for each month
## then selects only 2023 onwards 

def all_time_tickers(file_path):
    # Read the entire 10-K into a string
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        text = f.read()
    
    # Extract quoted ticker strings
    quoted_lists = re.findall(r'"([^"]+)"', text)
    
    # Convert each comma-separated string into a list of tickers
    ticker_lists = [entry.split(',') for entry in quoted_lists]
    
    # Extract the dates
    dates = re.findall(r'\d{4}-\d{2}-\d{2}', text)
    
    # Build a list of (date, ticker) tuples
    records = []
    for date, tickers in zip(dates, ticker_lists):
        for ticker in tickers:
            records.append((date, ticker))
    
    # Create the DataFrame
    df = pd.DataFrame(records, columns=['date', 'ticker'])
    
    df['date'] = pd.to_datetime(df['date'])
    df1 = df[df['date'] > '2021-12-31'].reset_index(drop=True)
    df1['date_clean'] = df1['date'].dt.to_period('M').astype(str)
    df1 = df1.replace("BF.B", "BF-B")
    df1 = df1.replace("BRK.B", "BRK-B")

    return df1

file_path = 'dates_tickers.txt'  # path will need to be chnaged
df1 = all_time_tickers(file_path) 





In [14]:

## if inner join works then company was in S&P 500 in that period
df3 = df1.merge(df2, on=['date_clean','ticker'] , how='inner')

In [18]:
df3['date'].sort_values(ascending=True)

0     2024-09-23
240   2024-09-23
239   2024-09-23
238   2024-09-23
237   2024-09-23
         ...    
732   2024-11-26
731   2024-11-26
730   2024-11-26
738   2024-11-26
748   2024-11-26
Name: date, Length: 749, dtype: datetime64[ns]

In [19]:
df3

,date,ticker,date_clean,cik,filing_type,filename,conformed_period_of_report,company_name,exchange
0,2024-09-23,AAPL,2024-09,0000320193,10-K,20241101_10-K_edgar_data_320193_0000320193-24-...,2024-09-28,Apple Inc.,Nasdaq
1,2024-09-23,ABBV,2024-09,0001551152,10-Q,20241104_10-Q_edgar_data_1551152_0001551152-24...,2024-09-30,AbbVie Inc.,NYSE
2,2024-09-23,ABNB,2024-09,0001559720,10-Q,20241107_10-Q_edgar_data_1559720_0001559720-24...,2024-09-30,"Airbnb, Inc.",Nasdaq
3,2024-09-23,ABT,2024-09,0000001800,10-Q,20241031_10-Q_edgar_data_1800_0001628280-24-04...,2024-09-30,ABBOTT LABORATORIES,NYSE
4,2024-09-23,ACGL,2024-09,0000947484,10-Q,20241107_10-Q_edgar_data_947484_0000947484-24-...,2024-09-30,ARCH CAPITAL GROUP LTD.,Nasdaq
...,...,...,...,...,...,...,...,...,...
744,2024-11-26,LW,2024-11,0001679273,10-Q,20241220_10-Q_edgar_data_1679273_0001679273-24...,2024-11-24,"Lamb Weston Holdings, Inc.",NYSE
745,2024-11-26,MU,2024-11,0000723125,10-Q,20241219_10-Q_edgar_data_723125_0000723125-24-...,2024-11-28,MICRON TECHNOLOGY INC,Nasdaq
746,2024-11-26,ORCL,2024-11,0001341439,10-Q,20241210_10-Q_edgar_data_1341439_0000950170-24...,2024-11-30,ORACLE CORP,NYSE
747,2024-11-26,PAYX,2024-11,0000723531,10-Q,20241219_10-Q_edgar_data_723531_0000950170-24-...,2024-11-30,PAYCHEX INC,Nasdaq


In [22]:
df1['ticker'].drop_duplicates

<bound method Series.drop_duplicates of 0           A
1         AAL
2         AAP
3        AAPL
4        ABBV
         ... 
22647     XYL
22648     YUM
22649     ZBH
22650    ZBRA
22651     ZTS
Name: ticker, Length: 22652, dtype: object>